WARNING
=========
<hr />

## Guys, excuse me for my poor English. it's a foreign language for me, and I will try hard to write in it as good as it is possible.

### If you want to see the implementation details, please refer to [my github repo](https://github.com/stolpa4/Coursera_Capstone/tree/master/Week3Assignment) with all the code.

<hr />

Step 1: Notebook created!!
--------------------------------------

<hr />

Step 2: Scrape the [wiki page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and create a dataframe
-----------------------------------------------------------------------------------------------------------------------------------

#### Import some useful staff:

In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup # Great web scraping library!

#### Download and parse web-page:

In [2]:
page_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_response = requests.get(page_link, timeout=5) # Use GET to obtain the response
page_content = BeautifulSoup(page_response.content, 'html.parser') # Use html parser to parse the page

#### Let's extract the table from the content and build a pandas DataFrame from it:

To make it easy for you I put all the overwhelming code to module `wiki_table_extractor`. You can check it out in the [project's github repo](https://github.com/stolpa4/Coursera_Capstone/tree/master/Week3Assignment).

For this task we will use a specially prepared class - `WikiTableExtractor`. Let's import it.

In [3]:
from wiki_table_extractor import WikiTableExtractor

This class' method `parse_table_from_page()` takes one argument - our `BeautifulSoup` instance and parses it to `Table` object (described below). Now we will proceed to instantiate the class and use the described method.

In [4]:
table_extractor = WikiTableExtractor()
table_extractor.parse_table_from_page(page_content)
extracted_table = table_extractor.table

Our `Table` object - `extracted_table` consists of two lists, one with table titles and another with table rows. We can use a method of this object - `as_dict_list()` to obtain a list of dictionaries with table rows.
Let's obtain the list and use it to create a pandas DataFrame object.

In [5]:
table_dict_list = extracted_table.as_dict_list()
canada_post_codes = pd.DataFrame(data=table_dict_list)
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (289, 3)


,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
9,Not assigned,Not assigned,M8A


#### Now it's time to refine our DataFrame!

At first, let's get rid of all these 'Not assigned' postal codes!  
Again, here the code is encapsulated in the module named `utils`. If you are interested in implementation details, please, check out [this module](https://github.com/stolpa4/Coursera_Capstone/blob/master/Week3Assignment/utils.py).

In [6]:
import utils

In [7]:
canada_post_codes = utils.filter_not_assigned_postcodes(canada_post_codes)
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (212, 3)


,Borough,Neighbourhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A
5,North York,Lawrence Manor,M6A
6,Queen's Park,Not assigned,M7A
7,Etobicoke,Islington Avenue,M9A
8,Scarborough,Rouge,M1B
9,Scarborough,Malvern,M1B


The next step is to name all neighborhoods that are 'Not assigned' (in fact, there is only one) after the corresponding Boroughs.  
To notice the result, pay attention to the row 6.

In [8]:
canada_post_codes = utils.name_not_assigned_neighborhoods(canada_post_codes)
canada_post_codes.head(10)

,Borough,Neighbourhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A
5,North York,Lawrence Manor,M6A
6,Queen's Park,Queen's Park,M7A
7,Etobicoke,Islington Avenue,M9A
8,Scarborough,Rouge,M1B
9,Scarborough,Malvern,M1B


Finally we need to combine the rows with the same post-code into one row with the neighborhoods separated with a comma.

In [9]:
canada_post_codes = utils.combine_rows_with_same_postcode(canada_post_codes)
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.tail(10)

DataFrame shape:  (103, 3)


,Postcode,Borough,Neighbourhood
93,M9A,Etobicoke,Islington Avenue
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery, Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


Now our data is structured properly and we can proceed to further steps.

<hr />

Step 4: Get the latitude and the longitude coordinates of each neighborhood 
--------------------------------------------------------------------------------------------------------------

#### let's use geocoder implicitly (code is in the util library, but in fact it's just a copy of the code from the assignment description)

In [10]:
latitude_column = []
longitude_column = []

for index, row in canada_post_codes.iterrows():
    latitude, longitude = utils.get_latitude_longitude(row['Postcode'])
    latitude_column.append(latitude)
    longitude_column.append(longitude)

Now, after we obtained lists with latitudes and longitudes of the neighborhoods, let's include them to our dataframe.

In [11]:
canada_post_codes['Latitude'] = latitude_column
canada_post_codes['Longitude'] = longitude_column
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",0,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",0,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,0
3,M1G,Scarborough,Woburn,0,0
4,M1H,Scarborough,Cedarbrae,0,0
5,M1J,Scarborough,Scarborough Village,0,0
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",0,0
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",0,0
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",0,0
9,M1N,Scarborough,"Birch Cliff, Cliffside West",0,0


In [12]:
WRONG_LATITUDES = canada_post_codes['Latitude'] == 0
WRONG_LONGITUDES = canada_post_codes['Longitude'] == 0

print('Wrong latitudes count: ', len(canada_post_codes[WRONG_LATITUDES]))
print('Wrong longitudes count: ', len(canada_post_codes[WRONG_LONGITUDES]))

Wrong latitudes count:  103
Wrong longitudes count:  103


Oh no! There must be some problem with geocoder library (or maybe I misused it, plesase check the code and let me know). Let's try again using CSV file with the coordinates

In [13]:
latitude_column = []
longitude_column = []

for index, row in canada_post_codes.iterrows():
    latitude, longitude = utils.get_latitude_longitude_from_csv(row['Postcode'])
    latitude_column.append(latitude)
    longitude_column.append(longitude)

In [14]:
canada_post_codes['Latitude'] = latitude_column
canada_post_codes['Longitude'] = longitude_column
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761
5,M1J,Scarborough,Scarborough Village,43.7447342,-79.2394761
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279292,-79.2620294
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111117,-79.2845772
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.2394761
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.2648481


Let's check that there is no zero coordinates:

In [15]:
WRONG_LATITUDES = canada_post_codes['Latitude'] == 0
WRONG_LONGITUDES = canada_post_codes['Longitude'] == 0

print('Wrong latitudes count: ', len(canada_post_codes[WRONG_LATITUDES]))
print('Wrong longitudes count: ', len(canada_post_codes[WRONG_LONGITUDES]))

Wrong latitudes count:  0
Wrong longitudes count:  0


<hr />

Final step: Exploring and clustering neighborhoods 
--------------------------------------------------------------------------

#### Let's work with only boroughs that contain the word Toronto.

In [16]:
WITH_TORONTO_WORD = canada_post_codes['Borough'].str.contains('Toronto')
canada_post_codes = canada_post_codes[WITH_TORONTO_WORD]
print('DataFrame shape: ', canada_post_codes.shape)
canada_post_codes.head(10)

DataFrame shape:  (38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6795571,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.6689985,-79.3155716
43,M4M,East Toronto,Studio District,43.6595255,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.7280205,-79.3887901
45,M4P,Central Toronto,Davisville North,43.7127511,-79.3901975
46,M4R,Central Toronto,North Toronto West,43.7153834,-79.4056784
47,M4S,Central Toronto,Davisville,43.7043244,-79.3887901
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.6895743,-79.3831599
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.6864123,-79.4000493


#### Explore Neighborhoods in Torronto

Let's replicate (and refactor a little bit) the `getNearbyVenues()`  function from the lab. I'm going to define it in the module `utils`. If you are interested in the code - [check GitHub repo](https://github.com/stolpa4/Coursera_Capstone/tree/master/Week3Assignment). Now we create a data frame `toronto_venues`.

In [26]:
toronto_venues = pd.DataFrame()
for index, row in canada_post_codes.iterrows():
    venues_list = utils.get_nearby_venues(row['Neighbourhood'],
                                          row['Latitude'],
                                          row['Longitude'])
    toronto_venues = toronto_venues.append(pd.DataFrame(venues_list))

print('DataFrame shape: ', toronto_venues.shape)
toronto_venues.head(10)

DataFrame shape:  (1692, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,The Beaches,43.6763574,-79.2930312,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,The Beaches,43.6763574,-79.2930312,Grover Pub and Grub,Pub,43.679181,-79.297215
2,The Beaches,43.6763574,-79.2930312,Starbucks,Coffee Shop,43.678798,-79.298045
3,The Beaches,43.6763574,-79.2930312,Upper Beaches,Neighborhood,43.680563,-79.292869
0,"The Danforth West, Riverdale",43.6795571,-79.352188,Pantheon,Greek Restaurant,43.677621,-79.351434
1,"The Danforth West, Riverdale",43.6795571,-79.352188,Dolce Gelato,Ice Cream Shop,43.677773,-79.351187
2,"The Danforth West, Riverdale",43.6795571,-79.352188,MenEssentials,Cosmetics Shop,43.677820,-79.351265
3,"The Danforth West, Riverdale",43.6795571,-79.352188,Messini Authentic Gyros,Greek Restaurant,43.677827,-79.350569
4,"The Danforth West, Riverdale",43.6795571,-79.352188,Cafe Fiorentina,Italian Restaurant,43.677743,-79.350115
5,"The Danforth West, Riverdale",43.6795571,-79.352188,Mezes,Greek Restaurant,43.677962,-79.350196


Let's check how many venues were returned for each neighbourhood

In [27]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


How many unique categories can be curated from all the returned venues?

In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


#### Analyze each neighbourhood

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

Print each neighbourhood along with 5 most common venues

In [36]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2        Beer Bar  0.04
3      Steakhouse  0.04
4  Farmers Market  0.04


----Brockton, Exhibition Place, Parkdale Village----
               venue  freq
0     Breakfast Spot  0.10
1        Coffee Shop  0.10
2               Café  0.10
3  Convenience Store  0.05
4                Gym  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Yoga Studio  0.06
1  Light Rail Station  0.06
2                Park  0.06
3          Comic Shop  0.06
4          Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport 